<a href="https://colab.research.google.com/gist/MarkCreightonQueens/0394572d9305f77344ec6e8873026897/machine-learning-training-bert-0_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random as r
r.seed(0)

In [ ]:
#installing tools
!pip install transformers 
!pip install wget
!pip install tika
!pip install ijson

     |████████████████████████████████| 757kB 601kB/s 
     |████████████████████████████████| 890kB 47.0MB/s 
     |████████████████████████████████| 1.1MB 42.1MB/s 
     |████████████████████████████████| 3.0MB 39.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=1b9859db0c4c3bc1b4c1f79fb41ff67f25eadd51be7fc022f37c59d23247c7df
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=02a51e5f0663f0f54149d0fb9c7f01d30e883fd4479f45eacbf12c536a9b735e
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget
  Created wheel for tika: filename=tika-1.24-cp36-none-any.whl size=32885 sha256=849fe0cada8a0c6702087c9ffbf5f0eb81d65ed09589c779ddcbf23e77908ef1
  Stored in directory: /root/.cache/pip/wheels/73/9c/f5/0b

In [ ]:
#Importing tools
import pandas as pd
import numpy as np
from tika import parser
%tensorflow_version 1.x
import tensorflow as tf
import torch
import os
import shutil
import glob
import nltk
nltk.download('punkt')
import wget
import urllib.request
import re
import gzip
from nltk.tokenize import sent_tokenize
import json

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

TensorFlow 1.x selected.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
#get GPU device name
device_name = tf.test.gpu_device_name()

if device_name == '/device:GPU:0':
  print('Found GPU at:{}'.format(device_name))
else:
  raise SystemError('GPU device not found')

Found GPU at:/device:GPU:0


In [ ]:
#Determining what GPU is available for use
if torch.cuda.is_available():
  device = torch.device("cuda")
  print ('There are %d GPU(s) available.' % torch.cuda.device_count())
  print('We will use the GPU:', torch.cuda.get_device_name(0))
else: 
  print('No GPU available, using CPU instead.')
  device = torch.device("CPU")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [ ]:
shutil.unpack_archive('/content/drive/My Drive/Thesis/yelp_review_full_csv.tgz')

In [ ]:
#Reading in the CSV. 

data_train_full = pd.read_csv('/content/yelp_review_full_csv/train.csv', header = None)
data_test = pd.read_csv('/content/yelp_review_full_csv/test.csv', header = None)

print(data_train_full.shape)
print(data_test.shape)

#Creating a smaller subset of the training dataset due to computational restrictions
data_train_1 = data_train_full[data_train_full[0] == 1].sample(50000, random_state=0)
data_train_2 = data_train_full[data_train_full[0] == 2].sample(50000, random_state=0)
data_train_3 = data_train_full[data_train_full[0] == 3].sample(50000, random_state=0)
data_train_4 = data_train_full[data_train_full[0] == 4].sample(50000, random_state=0)
data_train_5 = data_train_full[data_train_full[0] == 5].sample(50000, random_state=0)
frames = [data_train_1, data_train_2, data_train_3, data_train_4, data_train_5]
data_train = pd.concat(frames)
#randomizing the order of the observations
data_train = data_train.sample(frac=1, random_state=0)

print(data_train[0].unique())
print(data_train.shape)

#Reassigning values so that 1-3 = 0 and 4-5 = 1
data_train[0] = data_train[0].replace(1,0)
data_train[0] = data_train[0].replace(2,0)
data_train[0] = data_train[0].replace(3,0)
data_train[0] = data_train[0].replace(4,1)
data_train[0] = data_train[0].replace(5,1)

data_test[0] = data_test[0].replace(1,0)
data_test[0] = data_test[0].replace(2,0)
data_test[0] = data_test[0].replace(3,0)
data_test[0] = data_test[0].replace(4,1)
data_test[0] = data_test[0].replace(5,1)

print(data_train[0].unique())

(650000, 2)
(50000, 2)
[3 2 4 1 5]
(250000, 2)
[0 1]


In [ ]:
df_bert = pd.DataFrame({'id':range(len(data_train)),'label':data_train[0],'alpha':['a']*data_train.shape[0],'text':data_train[1].replace(r'\n', ' ', regex = True)})
print(df_bert.head())

df_bert_train, df_bert_dev = train_test_split(df_bert, test_size=0.01)

        id  label alpha                                               text
116433   0      0     a  The employees/students at EMCC are very friend...
308806   1      0     a  My kids and I went here to grab some take out ...
464848   2      1     a  This place is a 21 and up club that caters to ...
457220   3      0     a  This place is just NOT what it used to be.  Iv...
64604    4      0     a  Despite apprehensions of the 1st review here I...


In [ ]:
df_bert_test = pd.DataFrame({'id':range(len(data_test)), 'text': data_test[1].replace(r'\n', ' ', regex=True)})

df_bert_test.tail()

,id,text
49995,49995,Just wanted to write a review to chip in with ...
49996,49996,Great ambience. Great drinks. Great food. I lo...
49997,49997,I have been to the other Monks locations so I ...
49998,49998,Don't go here. I know you might want to try i...
49999,49999,Buffet was recently open after renovation so m...


In [ ]:
#Saving the dataframes to the required format. 

df_bert_train.to_csv('/content/drive/My Drive/Thesis/BERT/bert_data/train.tsv', sep='\t', index=False, header=False)
df_bert_dev.to_csv('/content/drive/My Drive/Thesis/BERT/bert_data/dev.tsv', sep='\t', index=False, header=False)
df_bert_test.to_csv('/content/drive/My Drive/Thesis/BERT/bert_data/test.tsv', sep='\t', index=False, header=True)

In [ ]:
#training the mode
os.chdir('/content/drive/My Drive/Thesis/BERT/bert-master') 
#commented out to stop accidental overwriting 
#!python run_classifier_0_1.py --task_name=cola --do_train=true --do_eval=true --data_dir='/content/drive/My Drive/Thesis/BERT/bert_data' --vocab_file='/content/drive/My Drive/Thesis/BERT/uncased_L-12_H-768_A-12/vocab.txt' --bert_config_file='/content/drive/My Drive/Thesis/BERT/uncased_L-12_H-768_A-12/bert_config.json' --init_checkpoint='/content/drive/My Drive/Thesis/BERT/uncased_L-12_H-768_A-12/bert_model.ckpt' --max_seq_length=128 --train_batch_size=32 --learning_rate=2e-5 --num_train_epochs=3.0 --output_dir='/content/drive/My Drive/Thesis/BERT/0_1/bert_output' --do_lower_case=True  --save_checkpoints_steps 16000

In [ ]:
#predicting sentiment
#os.chdir('/content/drive/My Drive/Thesis/BERT/bert-master')
#!python run_classifier_0_1.py --task_name=cola --do_predict=true --data_dir='/content/drive/My Drive/Thesis/BERT/bert_data' --vocab_file='/content/drive/My Drive/Thesis/BERT/uncased_L-12_H-768_A-12/vocab.txt' --bert_config_file='/content/drive/My Drive/Thesis/BERT/uncased_L-12_H-768_A-12/bert_config.json' --init_checkpoint='/content/drive/My Drive/Thesis/BERT/0_1/bert_output/model.ckpt-23203' - --max_seq_length=128 --output_dir='/content/drive/My Drive/Thesis/BERT/0_1/bert_output' 

In [ ]:
import csv
label_results=[]
pd.DataFrame()
with open('/content/drive/My Drive/Thesis/BERT/0_1/bert_output/test_results.tsv') as file:
    rd = csv.reader(file, delimiter="\t")
    for row in rd:
      data_1=[float(i) for i in row]
      label_results.append(data_1.index(max(data_1)))
df_results=pd.DataFrame()
df_results=pd.DataFrame(label_results)
df_results[0]

0        0
1        0
2        0
3        0
4        0
        ..
49995    0
49996    1
49997    1
49998    0
49999    0
Name: 0, Length: 50000, dtype: int64

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(data_test[0], df_results[0])

array([[27235,  2765],
       [ 2444, 17556]])

In [ ]:
from sklearn import metrics
print("Accuracy",metrics.accuracy_score(data_test[0], df_results[0]))
print("F1-Score",metrics.f1_score(data_test[0], df_results[0],average='weighted'))

Accuracy 0.89582
F1-Score 0.8959545138625271


In [ ]:
BERT_0_1_data_temp = pd.read_csv('/content/drive/My Drive/Thesis/Output_backup.csv')
print(BERT_0_1_data_temp.shape)

BERT_0_1_data = pd.DataFrame(columns=['id','text'])
BERT_0_1_data['id'] = BERT_0_1_data_temp['Meta Sentence ID']
BERT_0_1_data['text'] = BERT_0_1_data_temp['Sentences']
BERT_0_1_data
BERT_0_1_data.to_csv('/content/drive/My Drive/Thesis/BERT/bert_data/test.tsv', sep='\t', index=False, header=True)

(42083, 15)


In [ ]:
#os.chdir('/content/drive/My Drive/Thesis/BERT/bert-master')
!python run_classifier_0_1.py --task_name=cola --do_predict=true --data_dir='/content/drive/My Drive/Thesis/BERT/bert_data' --vocab_file='/content/drive/My Drive/Thesis/BERT/uncased_L-12_H-768_A-12/vocab.txt' --bert_config_file='/content/drive/My Drive/Thesis/BERT/uncased_L-12_H-768_A-12/bert_config.json' --init_checkpoint='/content/drive/My Drive/Thesis/BERT/0_1/bert_output/model.ckpt-23203' - --max_seq_length=128 --output_dir='/content/drive/My Drive/Thesis/BERT/0_1/bert_output' 





W0702 19:18:35.300803 139922567038848 module_wrapper.py:139] From run_classifier_0_1.py:784: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W0702 19:18:35.301045 139922567038848 module_wrapper.py:139] From run_classifier_0_1.py:784: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W0702 19:18:35.301502 139922567038848 module_wrapper.py:139] From /content/drive/My Drive/Thesis/BERT/bert-master/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0702 19:18:35.593874 139922567038848 module_wrapper.py:139] From run_classifier_0_1.py:808: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addon

In [ ]:
label_results=[]
pd.DataFrame()
with open('/content/drive/My Drive/Thesis/BERT/0_1/bert_output/test_results.tsv') as file:
    rd = csv.reader(file, delimiter="\t")
    for row in rd:
      data_1=[float(i) for i in row]
      label_results.append(data_1.index(max(data_1)))
df_results=pd.DataFrame()
df_results=pd.DataFrame(label_results)
df_results[0]


0.08483235510776323